In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import grammar as lt
from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed

2023-01-29 11:21:51.262505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-29 11:21:52.476918: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-29 11:21:52.476974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-29 11:21:52.476979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
with open('data/grammar.json', 'r') as fin:
    data = json.load(fin)

In [3]:
ctr = lt.err_counter(data)
ctr.most_common(5)

[('Unpaired symbol: ‘(’ seems to be missing', 8681),
 ('Unpaired symbol: ‘)’ seems to be missing', 8681),
 ('If specifying a range, consider using an en dash instead of a hyphen.',
  8345),
 ('This noun is normally spelled as one word.', 8226),
 ('The verb form ‘is’ does not seem to match the subject ‘imports’.', 8219)]

In [4]:
X = lt.filter_lt_messages(data, message='unpaired symbol')

In [6]:
qids = [c['question_id'] for c in X]

In [7]:
len(qids)

8681

In [8]:
len(set(qids))

8681

In [9]:
qids = sorted(qids)

In [11]:
df = []
for x in tqdm(pd.read_json('data/qa_captions.json', lines=True, chunksize=1_000_000)):
    x.set_index('question_id', verify_integrity=True, inplace=True)
    x = x[x.index.isin(qids)]
    df.append(x)

19it [01:24,  4.46s/it]


In [12]:
df = pd.concat(df, axis=0)
df.head()

,question_string,template,answer,type,template_id,caption
question_id,,,,,,
13196,What is the difference between the highest and...,compound,167970000,dot,2,"In the USD), the difference between the highes..."
22677,What is the difference between the highest and...,compound,248290000,dot,2,"In the USD), the difference between the highes..."
23132,What is the difference between the highest and...,compound,1320000,dot,2,"In the USD), the difference between the highes..."
24052,What is the difference between the highest and...,compound,890000,dot,2,"In the USD), the difference between the highes..."
35820,What is the difference between the highest and...,compound,690000,dot,2,"In the USD), the difference between the highes..."


In [19]:
x = df[df['template_id'] == 2]
x['caption'].head()

question_id
13196    In the USD), the difference between the highes...
22677    In the USD), the difference between the highes...
23132    In the USD), the difference between the highes...
24052    In the USD), the difference between the highes...
35820    In the USD), the difference between the highes...
Name: caption, dtype: object

In [68]:
import yaml
with open('qa_templates.yaml', 'r') as fin:
    cfg = yaml.safe_load(fin)
cfg = pd.DataFrame.from_records(cfg)
cfg.set_index('id', verify_integrity=True, inplace=True)
cfg.head()

,regex,template_header,caption_templates,type
id,,,,
1,Does the (?P<ylabel>.*) (?P<preposition>of|in)...,{% set metric = f'{ylabel} {preposition} {lege...,captions.randomize_1,binary
2,What is the (?P<yvalue>.*?) (?P<preposition>of...,,[The {{ yvalue }} {{ preposition }} {{ xvalue ...,retrieval
3,Is the sum of the (?P<ylabel1>.*) (?P<preposit...,"""{% set metric1 = f'{ylabel1} {preposition1} {...",captions.randomize_3,binary
4,What is the difference between the highest and...,,,retrieval
5,What is the difference between the highest and...,,,retrieval


In [22]:
import re

In [24]:
from plotqa import match_and_generate

In [26]:
row = x.iloc[0]
tmpl = cfg.loc[row['template_id']]
match_and_generate(row['question_string'], row['answer'], tmpl['regex'], tmpl['template_header'], tmpl['caption_templates'])

'The difference between the highest and the lowest amount of aid received(expressed in us$)  is 167970000.'

In [27]:
df.head()

,question_string,template,answer,type,template_id,caption
question_id,,,,,,
13196,What is the difference between the highest and...,compound,167970000,dot,2,"In the USD), the difference between the highes..."
22677,What is the difference between the highest and...,compound,248290000,dot,2,"In the USD), the difference between the highes..."
23132,What is the difference between the highest and...,compound,1320000,dot,2,"In the USD), the difference between the highes..."
24052,What is the difference between the highest and...,compound,890000,dot,2,"In the USD), the difference between the highes..."
35820,What is the difference between the highest and...,compound,690000,dot,2,"In the USD), the difference between the highes..."


In [107]:
df['new_caption'] = ''
# df.set_index('question_id', verify_integrity=True, inplace=True)
for qid, row in tqdm(df.iterrows()):
    tmpl = cfg.loc[row['template_id']]
    df.loc[qid, 'new_caption'] = match_and_generate(row['question_string'], row['answer'], tmpl['regex'], tmpl['template_header'], tmpl['caption_templates'])

8681it [00:04, 2126.82it/s]


In [108]:
df.head()

,question_string,template,answer,type,template_id,caption,new_caption
question_id,,,,,,,
13196,What is the difference between the highest and...,compound,167970000,dot,2,The difference between the highest and the lo...,167970000 is the difference between the highes...
22677,What is the difference between the highest and...,compound,248290000,dot,2,The difference between the highest and the low...,The difference between the highest and the low...
23132,What is the difference between the highest and...,compound,1320000,dot,2,The difference between the highest and the sec...,The difference between the highest and the sec...
24052,What is the difference between the highest and...,compound,890000,dot,2,The difference between the highest and the sec...,890000 is the difference between the highest a...
35820,What is the difference between the highest and...,compound,690000,dot,2,The difference between the highest and the se...,690000 is the difference between the highest a...


In [109]:
df['caption'] = df.pop('new_caption')

In [110]:
df.reset_index(inplace=True)
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(**row) for _, row in df.iterrows())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-01-29 12:32:22.228497: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-29 12:32:22.228798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-29 12:32:22.228809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-29 12:32:22.240581: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [111]:
ctr = lt.err_counter(res)
ctr.most_common(5)

[('It seems like there are too many consecutive spaces here.', 7988),
 ('Possible typo: you repeated a whitespace', 2729),
 ('Unpaired symbol: ‘(’ seems to be missing', 1456),
 ('Unpaired symbol: ‘)’ seems to be missing', 1456),
 ('A determiner may be missing.', 632)]

In [112]:
x = lt.filter_lt_messages(res, message='unpaired symbol')

In [113]:
qids = [c['question_id'] for c in x]
len(set(qids))

1456

In [114]:
df.set_index('question_id', verify_integrity=True, inplace=True)

In [115]:
xdf = df.loc[qids]
xdf['template_id'].value_counts()

42    1456
Name: template_id, dtype: int64

In [116]:
xdf[['template_id', 'question_string', 'caption']].sample().to_dict(orient='records')

[{'template_id': 42,
  'question_string': 'In the year 2009, what is the difference between the Payments(% of exports of goods, services and primary income) in Mauritania and Payments(% of exports of goods, services and primary income) in Tonga ?',
  'caption': '  In 2009, the primary income) in Tonga  is greater than the Payments(% of exports of goods, services and primary income) in Mauritania and Payments(% of exports of goods, services by -1.2198.'}]

In [118]:
pat = "In the year (?P<year>.*), what is the difference between the (?P<ylabel1>.*?) (?P<preposition1>of|in) (?P<legendlabel1>.*?) and (?P<ylabel2>.*?) (?P<preposition2>of|in) (?P<legendlabel2>.*?)\\s?\\?$"
s = "In the year 2009, what is the difference between the Payments(% of exports of goods, services and primary income) in Mauritania and Payments(% of exports of goods, services and primary income) in Tonga ?"
re.search(pat, s).groupdict()

{'xvalue': '2009',
 'ylabel1': 'Payments(%',
 'preposition1': 'of',
 'legendlabel1': 'exports of goods, services',
 'ylabel2': 'primary income)',
 'preposition2': 'in',
 'legendlabel2': 'Mauritania and Payments(% of exports of goods, services and primary income) in Tonga'}

In [102]:
# Confusion between template 44 and 37

In [103]:
tid44 = df[df['template_id'] == 44]
tid44['question_string'].str.contains('and that in').sum(), tid44.shape

(5951, (5951, 6))

In [104]:
df.loc[df['template_id'] == 44, 'template_id'] = 37

In [119]:
df.shape

(8681, 6)

In [121]:
df.reset_index().to_json('data/unp_sym.json', orient='records', lines=True)